# Use Cohere Command R or Cohere Command R+ with LangChain and Azure AI Search to Answer Questions Using Your Data

You can use Cohere Command R or Cohere Command R+ models deployed in Azure AI and Azure ML with `langchain` to create advanced retrieval augmented generation (RAG) pipelines.

> Review the [documentation](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/deploy-models-cohere-command) for the Cohere family of models at for AI Studio and for ML Studio for details on how to provision inference endpoints, regional availability, pricing and inference schema reference.

## Prerequisites

Before we start, there are certain steps we need to take to deploy the models:

* Register for a valid Azure account with subscription 
* Make sure you have access to [Azure AI Studio](https://learn.microsoft.com/en-us/azure/ai-studio/what-is-ai-studio?tabs=home)
* Create a project and resource group
* Select `Cohere Command R` or `Cohere Command R+`.

    > Notice that some models may not be available in all the regions in Azure AI and Azure Machine Learning. On those cases, you can create a workspace or project in the region where the models are available and then consume it with a connection from a different one. To learn more about using connections see [Consume models with connections](https://learn.microsoft.com/en-us/azure/ai-studio/concepts/connections)

* Deploy with "Pay-as-you-go"
* Follow the same steps for `Cohere-embed-v3-english`

Once deployed successfully, you should be assigned for an API endpoint and a security key for inference.

For more information, you should consult Azure's official documentation [here](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/deploy-models-cohere-command) for model deployment and inference.

### Services

You will need to ensure the following services have been created in your Azure environment:
* Ensure you have created a search service. This can be done in the `Azure Portal` and more instructions can be found here: https://learn.microsoft.com/en-us/azure/search/search-what-is-azure-search **NOTE: You do not need to create the index, this will be done below**

* Create a Cohere `Embed` and `Command` endpoint in the `Azure AI Studio`. Instructions can be found here: https://learn.microsoft.com/en-us/azure/ai-studio/how-to/deploy-models-cohere-command 

## Example

The following example demonstrate how to create a RAG workflow that uses a `Cohere Command R` or `Cohere Command R+` model deployed in Azure AI and Azure ML. We will also leverage Azure AI Search to store our documents along with LangChain to orchestrate the process

### Install Dependencies

In [ ]:
! pip install --quiet langchain langchain-cohere azure-search-documents azure-identity

In [139]:
import os
from azure.core.credentials import AzureKeyCredential
from langchain_community.vectorstores.azuresearch import AzureSearch
from langchain_cohere import ChatCohere
from langchain_cohere import CohereEmbeddings
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain.schema import HumanMessage


## Constants
We will set the values for the keys and models that we will use in our RAG pipeline

Let's create instances of our Chat and Embed models deployed in Azure AI or Azure ML. Use the `langchain_cohere` package and configure it as follows:

- `embed_endpoint` and `command_endpoint`: Use the endpoint URL from your deployment. Include `/v1` at the end of the endpoint URL.

In [ ]:
# set constants
search_service_endpoint = "https://<resource>.search.windows.net"
key_credential = AzureKeyCredential("AZURE_SEARCH_ADMIN_KEY")
index_name = "AZURE_SEARCH_INDEX"  # this index does not have to be created yet

# Embed
azure_cohere_embed_endpoint = "https://<endpoint>.<region>.inference.ai.azure.com/v1"
azure_cohere_embed_key = "<key>"

# Command
azure_cohere_command_endpoint = "https://<endpoint>.<region>.inference.ai.azure.com/v1"
azure_cohere_command_key = "<key>"

### AI Search Index

We need to set up the index in our `Azure AI Search`

We will use the `LangChain` `AzureSearch` package as well as the `CohereEmbeddings` package, which will serve as the embedding model for our index

In [ ]:
# Set up the embedding model to be used in the vector index
embed_model = CohereEmbeddings(
    base_url=azure_cohere_embed_endpoint,
    cohere_api_key=azure_cohere_embed_key,
)

In [ ]:
# create the vector store using AzureSearch
vector_store = AzureSearch(
    azure_search_endpoint=search_service_endpoint,
    azure_search_key=key_credential,
    index_name=index_name,
    embedding_function=embed_model,
    semantic_configuration_name="default",
)

Lets create some sample data to add to our index

In [ ]:
docs_to_index = [
    {
        "hotelId": "1",
        "hotelName": "Fancy Stay",
        "description": "Best hotel in town if you like luxury hotels.",
        "category": "Luxury",
    },
    {
        "hotelId": "2",
        "hotelName": "Roach Motel",
        "description": "Cheapest hotel in town. Infact, a motel.",
        "category": "Budget",
    },
    {
        "hotelId": "3",
        "hotelName": "EconoStay",
        "description": "Very popular hotel in town.",
        "category": "Budget",
    },
    {
        "hotelId": "4",
        "hotelName": "Modern Stay",
        "description": "Modern architecture, very polite staff and very clean. Also very affordable.",
        "category": "Luxury",
    },
    {
        "hotelId": "5",
        "hotelName": "Secret Point",
        "description": "One of the best hotel in town. The hotel is ideally located on the main commercial artery of the city in the heart of New York.",
        "category": "Boutique",
    },
]

Finally, we will add the texts into our vector index. This will automatically embed the field we choose (description in this case) and attach the associated metadata

In [ ]:
vector_store.add_texts(
    texts=[d["description"] for d in docs_to_index], metadatas=docs_to_index
)

We will create a langchain retriever with our newly-populated Azure AI vector store

In [ ]:
retriever = vector_store.as_retriever()

In [ ]:
retriever.get_relevant_documents("Best luxury hotel in town", top_k=3)

### Create our Chat function
Next, we will create a basic chat function using the `ChatCohere` class

Cohere's `chat endpoint` can accept documents directly and will return a grounded answer that includes citations against those documents.

No prompt is needed since the `Cohere` model will automatically use a RAG prompt when documents are passed in.

Because we are using `LangChain's expression language (LCEL)`, we will also wrap our function at the end with a `RunableLambda` function. Learn more about LCEL here: https://python.langchain.com/docs/expression_language/get_started/

In [ ]:
def ask(inputs):
    """
    Ask a question to the chatbot, expecting a dictionary with 'question' and 'documents'.

    Args:
        inputs (dict): A dictionary containing 'question' and 'documents'.

    Returns:
        str: The response from the chatbot.
    """
    question = inputs["question"]
    documents = inputs["documents"]

    documents = [d.metadata for d in documents]

    chatbot = ChatCohere(
        base_url=azure_cohere_command_endpoint,
        cohere_api_key=azure_cohere_command_key,
    )

    response = chatbot(messages=[HumanMessage(content=question)], documents=documents)
    return response


ask = RunnableLambda(ask)

## Create the Chain
We can now create the chain by chaining the retriever and the chat function together

In [ ]:
# create the answer chain using our search function and chat function
answer_chain = {"documents": retriever, "question": RunnablePassthrough()} | ask

## Run our chain
You can now run the full chain and see the response as an `AIMessage` object. This will contain the models answer along with citations from the documents retrieved from `Azure AI Search`

In [ ]:
answer_chain.invoke("luxury hotel")

## Aditional resources

Here are some additional reference:

* [Plan and manage costs (marketplace)](https://learn.microsoft.com/azure/ai-studio/how-to/costs-plan-manage#monitor-costs-for-models-offered-through-the-azure-marketplace)
                                        
* [Cohere examples with LangChain](https://docs.cohere.com/docs/cohere-and-langchain)      